# Case

In [1]:
import numpy as np
import pandas as pd

from typing import Dict

import andes
import ams

In [2]:
%matplotlib inline

In [3]:
!ams misc --version

Python   3.12.0
ams      1.0.7
andes    1.9.3.post6+g518882a0
numpy    2.0.2
cvxpy    1.6.0
solvers  CLARABEL, ECOS, ECOS_BB,
         GUROBI, HIGHS, MOSEK, OSQP,
         PIQP, SCIP, SCIPY, SCS


In [4]:
andes.config_logger(stream_level=30)
ams.config_logger(stream_level=20)

In [5]:
mpc5 = ams.io.matpower.m2mpc("./Texas2k/Texas2k_case5.m")

Tune the parameters

In [6]:
gencost = np.concatenate((mpc5['gencost'][:, 0:4],
                          np.zeros((mpc5["gencost"].shape[0], 4))),
                         axis=1)

gencost[:, 3] = 2  # type 2 gencost
gencost[:, 5] = 0.0  # c2
gencost[:, 6] = mpc5['gencost'][:, 5]  # c1
gencost[:, 7] = mpc5['gencost'][:, 5]  # c0

# scale up some line thermal limits
line_id = np.array([343,  344, 2324])
branch = mpc5['branch'].copy()
branch[:, 5] = 1.1 * mpc5['branch'][:, 5]

mpc_out = mpc5.copy()
mpc_out['gencost'] = gencost
mpc_out['branch'] = branch

In [7]:
sp = ams.system.System()
ams.io.matpower.mpc2system(mpc_out, sp)

sp.setup()

Zero Line parameters detected, adjusted to default values: rate_b, rate_c, amax, amin.
System set up in 0.0074 seconds.


True

Export to JSON format.

In [8]:
# ams.io.json.write(sp, './Texas2k/Texas2k_case5.json', overwrite=True)

In [9]:
sp.DCOPF.run(solver='PIQP')

Building system matrices
Parsing OModel for <DCOPF>
Evaluating OModel for <DCOPF>
Finalizing OModel for <DCOPF>
<DCOPF> initialized in 0.5081 seconds.
<DCOPF> solved as optimal in 0.1942 seconds, converged in 14 iterations with PIQP.


True

In [10]:
dyr = andes.io.psse._read_dyr_dict("./Texas2k/Texas2k.dyr")

In ANDES v1.9.3, some unsupported models can be replaced with supported models.

[GGOV1](https://www.powerworld.com/WebHelp/Content/TransientModels_HTML/Governor%20GGOV1%20and%20GGOV1D.htm) -> TGOV1

[PSS2A](https://www.powerworld.com/WebHelp/Content/TransientModels_HTML/Stabilizer%20PSS2A.htm) -> [ST2CUT](https://www.powerworld.com/WebHelp/Content/TransientModels_HTML/Stabilizer%20ST2CUT.htm)

[GENSAL](https://www.powerworld.com/WebHelp/Content/TransientModels_HTML/Generator%20Model%20GENSAL.htm) -> GENROU

In [11]:
sa = andes.system.System()

In [12]:
def write_dict_dyr(dyr_dict: Dict[str, pd.DataFrame], outfile: str):
    """
    Write a dictionary of dynamic models to a DYR file.

    This function takes a dictionary where the keys are model names (e.g., 'REGCA1', 'REECA1') 
    and the values are pandas DataFrames containing the model parameters. It writes the data 
    to a DYR file in the correct format, ensuring the following:
    - The first column (IBUS) is an integer representing the bus index.
    - The second column is the model name, enclosed in single quotes.
    - The third column (ID) is an integer representing the device index.
    - Remaining columns are the model parameters, written as space-separated values.
    - Each line ends with a trailing slash ('/').

    Models 'Toggle' are skipped during the writing process.

    Parameters:
    -----------
    dyr_dict : Dict[str, pd.DataFrame]
        A dictionary where each key is a model name and each value is a pandas DataFrame 
        containing the parameters for that model. The DataFrame must have the following structure:
        - Column 0: IBUS (integer)
        - Column 1: ID (integer)
        - Columns 2+: Model parameters (various types).
    
    outfile : str
        The path to the output DYR file where the formatted data will be written.

    Returns:
    --------
    None
        The function writes the data to the specified file and prints a success message.

    Notes:
    ------
    - The function skips models with the name 'Toggle' and prints a message for each skipped model.
    - A blank line is not added between models in the output file.
    - Ensure the input DataFrames are properly formatted to avoid runtime errors.

    Example:
    --------
    >>> dyr_dict = {
    ...     'REGCA1': pd.DataFrame([
    ...         [1004, 1, 1.0, 0.01, 10, 0.9, 0.5],
    ...         [1006, 1, 1.0, 0.02, 10, 0.9, 0.5]
    ...     ]),
    ...     'REECA1': pd.DataFrame([
    ...         [1004, 1, 0, 0, 1, 1, 1],
    ...         [1006, 1, 0, 0, 1, 1, 1]
    ...     ])
    ... }
    >>> write_dict_dyr(dyr_dict, './output.dyr')
    Data successfully written to ./output.dyr
    """
    with open(outfile, 'w') as f:
        for model_name, df in dyr_dict.items():
            if model_name in ['Toggle']:
                print(f"Skipped {model_name}")
                continue
            for _, row in df.iterrows():
                ibus = int(row.iloc[0])  # format IBUS as integer
                model = f"'{model_name}'"  # model name
                device_id = int(row.iloc[1])  # format ID as integer
                # format the rest parameters
                params = ' '.join(map(str, row.iloc[2:].values))
                # Combine all parts into a formatted line
                formatted_line = f"{ibus} {model} {device_id} {params} /"
                # Write the formatted line to the file
                f.write(formatted_line + '\n')

    print(f"Data successfully written to {outfile}")

In [13]:
# Duplicate supported models first
dyr_out = dict()

for mname in dyr.keys():
    if mname in ['GGOV1', 'PSS2A', 'GENSAL']:
        continue
    dyr_out[mname] = dyr[mname].copy()

# GENSAL -> GENROU
GENROU_converted = dyr['GENSAL'].copy()
GENROU_converted[14] = sa.GENROU.S10.default
GENROU_converted[15] = sa.GENROU.S12.default
dyr_out['GENROU'] = pd.concat([dyr_out['GENROU'], GENROU_converted],
                              axis=0, ignore_index=True)

# PSS2A -> ST2CUT, there are 22 columns in ST2CUT
ST2CUT = pd.DataFrame()

ST2CUT[0] = dyr['PSS2A'][0]  # IBUS
ST2CUT[1] = dyr['PSS2A'][1]  # ID

ST2CUT[2] = dyr['PSS2A'][2]  # M, IC1 <- M, ICS1
ST2CUT[3] = dyr['PSS2A'][3]  # M+1, IB1 <- M+1, REMBUS1
ST2CUT[4] = dyr['PSS2A'][4]  # M+2, IB2 <- M+2, ICS2
ST2CUT[5] = dyr['PSS2A'][5]  # M+3, IB3 <- M+3, REMBUS2

ST2CUT[6] = 1  # J, K1 <- 1
# J+1, K2 <- J+6, Ks2 * J+7, Ks3
ST2CUT[7] = dyr['PSS2A'][14] * dyr['PSS2A'][15]
ST2CUT[8] = dyr['PSS2A'][10]  # J+2, T1 <- J+2, T6
ST2CUT[9] = dyr['PSS2A'][13]  # J+3, T2 <- J+5, T7
ST2CUT[10] = sa.ST2CUT.T3.default  # J+4, T3 <- default
ST2CUT[11] = sa.ST2CUT.T4.default  # J+5, T4 <- default
ST2CUT[12] = dyr['PSS2A'][19]  # J+6, T5 <- J+11, T1
ST2CUT[13] = dyr['PSS2A'][20]  # J+7, T6 <- J+12, T2
ST2CUT[14] = dyr['PSS2A'][21]  # J+8, T7 <- J+13, T3
ST2CUT[15] = dyr['PSS2A'][22]  # J+9, T8 <- J+14, T4
ST2CUT[16] = sa.ST2CUT.T9.default  # J+10, T9 <- default
ST2CUT[17] = ST2CUT[16]  # J+11, T10 <- copy of T9
ST2CUT[18] = dyr['PSS2A'][23]  # J+12, LSMAX <- J+15, VSTMAX
ST2CUT[19] = dyr['PSS2A'][24]  # J+13, LSMIN <- J+16, VSTMIN
ST2CUT[20] = sa.ST2CUT.VCU.default  # J+14, VCU <- default
ST2CUT[21] = sa.ST2CUT.VCL.default  # J+15, VCL <- default
dyr_out['ST2CUT'] = ST2CUT

# GGOV1 -> TGOV1, there are 9 columns in TGOV1
TGOV1_converted = pd.DataFrame()
TGOV1_converted[0] = dyr['GGOV1'][0]  # IBUS
TGOV1_converted[1] = dyr['GGOV1'][1]  # ID
TGOV1_converted[2] = dyr['GGOV1'][2]  # J, R <- J, r
TGOV1_converted[3] = dyr['GGOV1'][16]  # J+1, T1 <- J+15, Teng
TGOV1_converted[4] = sa.TGOV1.VMAX.default  # J+2, T2 <- J+16, Te
TGOV1_converted[5] = sa.TGOV1.VMIN.default  # J+2, T2 <- J+16, Te
TGOV1_converted[6] = dyr['GGOV1'][15]  # J+4, T2 <- J+14, Tc
TGOV1_converted[7] = dyr['GGOV1'][14]  # J+5, T3 <- J+13, Tb
TGOV1_converted[8] = dyr['GGOV1'][15]  # J+6, Dt <- J+20, Dm
dyr_out['TGOV1'] = TGOV1_converted

In [14]:
# NOTE: hard code to remove folowing renewable generation involved devices 
# if they connect at bus 5394, 5395, 7095, 7099

for mname, df in dyr_out.items():
    if mname in ['REGCA1', 'REECA1', 'REPCA1', 'WTPTA1', 'WTTQA1', 'WTARA1']:
        rows_to_drop = df[df[0].isin([5394, 5395, 7095, 7099])].index
        df.drop(rows_to_drop, inplace=True)
        dyr_out[mname] = df.reset_index(drop=True)

In [ ]:
# # Write the original DYR file to sort it, without changing anything
# write_dict_dyr(dyr, './Texas2k/Texas2k_case5_sorted.dyr')

# Write the modified DYR file
write_dict_dyr(dyr_out, './Texas2k/Texas2k_case5_mod.dyr')

Data successfully written to ./Texas2k/Texas2k_case5_sorted.dyr
Data successfully written to ./Texas2k/Texas2k_case5_mod.dyr


In [18]:
# addfile="Texas2k/Texas2k_case5_mod_syngen_plus.dyr"

sa0 = andes.load(
    "Texas2k/Texas2k_v33.raw",
    addfile="Texas2k/Texas2k_case5_mod.dyr",
    setup=False,
)

Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero parameter GENROU.M corrected to 6
Non-zero pa

In [19]:
gr = sa0.GENROU.as_df().copy()

AttributeError: 'NoneType' object has no attribute 'GENROU'

Test note:
- REGCA1, ok
- REECA1, removed three devices said there is no corresponding unfined REGCA1
- REPCA1, same as above
- WTPTA1, same as above

REECA1


5394, 5395, 7095, 7099

REPCA1

604 -> 905; X

604 -> 750; X

604 -> 700; X

604 -> 650;

Others and SynGen, REGCA1, REECA1

In [ ]:
dyr_out['REPCA1']

In [ ]:
sa0.Line.as_df().head(3)

In [ ]:
sa0.REPCA1.as_df()